In [1]:
import numpy as np
import pandas as pd

from os import scandir
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm

In [2]:
rutaASI='C:\\Users\\boliv\\Desktop\\Ejercicio_Python\\Codigo_MCLJ_PCA\\Datos_SAC_LJMC\\ASI'
rutaCO= "C:\\Users\\boliv\\Desktop\\Ejercicio_Python\\Codigo_MCLJ_PCA\\Datos_SAC_LJMC\\CO"
# función que regresa todos los archivos en la ruta path
def dimeArchivos(path): 
    return [obj.name for obj in scandir(path) if obj.is_file()]

#Listas con los nombres de los archivos en la ruta dada
ListaArchASI = dimeArchivos(rutaASI)
ListaArchCO  = dimeArchivos(rutaCO)

#print(len(ListaArchASI))
#print(ListaArchASI)

#Crea lista con las primeras dos letras de cada archivo, correspodientes a los nombres de los sujetos
Name_suj_ASI = [x[0:2] for x in ListaArchASI]
Name_suj_CO  = [x[0:2] for x in ListaArchCO]

#print(Name_suj)

#Crea dict para eliminar duplicados y los convierte en lista
Name_suj_ASI = list(dict.fromkeys(Name_suj_ASI)) # ['BG','CR','EN','GF','GV','IF','IM','JE','JU','JV','LM','MI','RB','RR','VC','VV']
Name_suj_CO = list(dict.fromkeys(Name_suj_CO))   # ['AG','AS','AZ','BE','CV','EG','GM','GV','LO','LP','LR','MN','MS','PA','TR']

#print(Name_suj_ASI,type(Name_suj_ASI))
#print(Name_suj_CO, type(Name_suj_CO))

#Selecciona aleatoriamente 12 sujetos para crear los conjuntos de prueba
ASI_12 = sorted(random.sample(Name_suj_ASI,12))
CO_12  = sorted(random.sample(Name_suj_CO, 12))

#Crea conjuntos de entrenamiento con los sujetos no seleccionados en los conjuntos de prueba
ASI_4 = [x for x in Name_suj_ASI if x not in ASI_12]
CO_3  = [x for x in Name_suj_CO  if x not in CO_12]

In [3]:
print('ASI_12: ', ASI_12, '\n', 'CO_12: ', CO_12, '\n', 'ASI_4: ', ASI_4, '\n', 'CO_3: ', CO_3)

ASI_12:  ['BG', 'EN', 'GF', 'GV', 'IF', 'IM', 'JU', 'JV', 'LM', 'MI', 'VC', 'VV'] 
 CO_12:  ['AG', 'AS', 'AZ', 'BE', 'CV', 'EG', 'GM', 'LO', 'LP', 'MN', 'MS', 'TR'] 
 ASI_4:  ['CR', 'JE', 'RB', 'RR'] 
 CO_3:  ['GV', 'LR', 'PA']


In [4]:
#Lista de las rutas de los archivos por conjunto (train/test)
Train_ASI = [rutaASI+"\\"+x for x in ListaArchASI if x[0:2] in     ASI_12]
Test_ASI  = [rutaASI+"\\"+x for x in ListaArchASI if x[0:2] not in ASI_12]
Train_CO  = [rutaCO+"\\"+x  for x in ListaArchCO  if x[0:2] in     CO_12]
Test_CO   = [rutaCO+"\\"+x  for x in ListaArchCO  if x[0:2] not in CO_12]

#for i in range (len(Test_ASI)):
#    print(Test_ASI[i])

In [5]:
# Arma arreglo de natriz SAC
ASI_train=[]
CO_train=[]

asi_clase = np.array ([-1])
co_clase = np.array ([1])


for i in Train_ASI: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, asi_clase))
    ASI_train = np.concatenate((ASI_train,array_sac))

ASI_train = ASI_train.reshape(len(ASI_12)*14,92)

for i in Train_CO: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, co_clase))
    CO_train = np.concatenate((CO_train,array_sac))

CO_train = CO_train.reshape(len(CO_12)*14,92)

#print(Arreglo_ASI[209])
        
SVM_train = np.concatenate((ASI_train,CO_train), axis=0)
print(SVM_train.shape)


(336, 92)


In [6]:
# Arma arreglo de natriz SAC
ASI_test=[]
CO_test=[]

for i in Test_ASI: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, asi_clase))
    ASI_test = np.concatenate((ASI_test,array_sac))

ASI_test = ASI_test.reshape(len(ASI_4)*14,92)

for i in Test_CO: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, co_clase))
    CO_test = np.concatenate((CO_test,array_sac))

CO_test = CO_test.reshape(len(CO_3)*14,92)

#print(Arreglo_ASI[209])
        
SVM_test = np.concatenate((ASI_test,CO_test), axis=0)
print(SVM_test.shape)


(98, 92)


In [7]:
X_train = SVM_train[:, 0:91]
X_test = SVM_test[:, 0:91]
y_train = SVM_train[:,-1]
y_test = SVM_test[:, -1]

In [8]:
print('Train_Set: ',X_train.shape, y_train.shape)

print('Test_Set: ',X_test.shape, y_test.shape)

Train_Set:  (336, 91) (336,)
Test_Set:  (98, 91) (98,)


In [9]:
#Intervalo de parámetros
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}

In [10]:
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1)

In [11]:
# Se buscan los mejores parámetros para los conjuntos de entrenamiento
clf_grid.fit(X_train, y_train)

print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

C:\Users\boliv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters:
 {'C': 100, 'gamma': 0.01}
Best Estimators:
 SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    0.6s finished


In [12]:
#Se ajusta el clasificador con los mejores parámetros encontrados
clf = svm.SVC(kernel='rbf', C=100, gamma=0.01)
clf.fit(X_train,y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
# Se realizan las predicciones en conjuntos de prueba
clf_predictions = clf.predict(X_test)
print("Accuracy: {0:.2F}%".format(clf.score(X_test, y_test) * 100 ))

Accuracy: 89.80%
